# E KNN and K-means CIFAR


transform cifar-10 to grayscale.

- Does knn work similarly good?
- Does k-means work similarly good?
- Demonstrate this similar to B, C and D


In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import f1_score
import scipy.spatial.distance
import seaborn as sns
from sklearn.manifold import TSNE

plt.gray()
import skimage as ski

In [2]:
cifar = tf.keras.datasets.cifar10.load_data()[0] 
X_train, Y_train = cifar[0], cifar[1]
X_test, Y_test = cifar[0], cifar[1]

In [3]:
#convert CIFAR 10 to grayscale
X_train = ski.color.rgb2gray(X_train)
X_test = ski.color.rgb2gray(X_test)

In [4]:
#flatten data
n_train = X_train.shape[0]
n_test = X_test.shape[0]

m = X_train.shape[1]**2

X_train = X_train.reshape([n_train, m])
X_test = X_test.reshape([n_test, m])

In [5]:
X_train = X_train.reshape([n_train, m])
X_test = X_test.reshape([n_test, m])

idx = np.random.randint(n_train, size=10000)
x_train_sample = X_train[idx]
y_train_sample = Y_train[idx]

idx = np.random.randint(n_test, size=100)
x_test_sample = X_test[idx]
y_test_sample = Y_test[idx]

karray = [2, 4, 8]

In [13]:
results = []
def endresult():
    for k in karray:
        result_array = []
        dists = scipy.spatial.distance.cdist(x_train_sample, x_test_sample, metric='euclid')
        idx_nearest = np.argpartition(dists, k, axis = 0)[:k]
        nearest_dists = np.take(x_train_sample, idx)
    
        pred = y_train_sample[idx_nearest]
        pred = np.ndarray.transpose(pred)
        pred = pred.reshape(100,k)
        actuals = y_test_sample
        prediction = np.array([])
        
        for row in pred:
            count = np.bincount(row)
            prediction = np.append(prediction, int(np.argmax(count)))
    
        prediction = prediction.astype(int)
        '''We use 1-f1_score since it is an better indicator of quality than the error rate.'''
        calcf1score = f1_score(actuals, prediction, average = 'micro')
        print ("The error rate for KNN with k = {} on this Dataset is {}".format(k, (1 - calcf1score)))

        for i in range(10):
            indices = np.where(actuals == i)
            prediction = prediction.reshape(100,1)
            f1 = 1-(f1_score(actuals[indices], prediction[indices], average = 'micro'))
            result_array.append(f1)
            
        results.append(result_array)
               
endresult()

The error rate for KNN with k = 2 on this Dataset is 0.7999999999999999
The error rate for KNN with k = 4 on this Dataset is 0.77
The error rate for KNN with k = 8 on this Dataset is 0.81


In [15]:
pd.DataFrame(np.array(results).T, columns=["k = 2", "k = 4", "k = 8"])

,k = 2,k = 4,k = 8
0,0.636364,0.363636,0.545455
1,0.833333,1.000000,1.000000
2,0.600000,0.800000,1.000000
3,0.888889,1.000000,1.000000
4,0.666667,0.777778,0.666667
5,0.857143,0.857143,0.857143
6,0.923077,0.923077,0.846154
7,0.769231,0.538462,0.769231
8,0.800000,0.800000,0.600000
9,0.866667,0.800000,0.866667


As we can see KNN works very badly with this converted Dataset. 